In [ ]:
# Prérequis : Installer les modules présents dans le notebook recap

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Utiliser les embeddings d'HuggingFace
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [ ]:
# Comparer les embeddings d'une question et des metadata

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("bdconsignes.txt")
pages_txt=loader.load()

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

In [ ]:
md_header_splits = markdown_splitter.split_text(pages_txt[0].page_content)

In [ ]:
metadata_docs=[]
for i in range(len(md_header_splits)):
    metadata_inter=""
    if any(item in list(md_header_splits[i].metadata.keys()) for item in ['Header 2','Header 3']) is False:
        metadata_inter=metadata_inter+md_header_splits[i].metadata['Header 1']
    if 'Header 2' in list(md_header_splits[i].metadata.keys()):
        metadata_inter=metadata_inter+md_header_splits[i].metadata['Header 2']
    if 'Header 3' in list(md_header_splits[i].metadata.keys()):
        metadata_inter=metadata_inter+'. '+md_header_splits[i].metadata['Header 3']
    metadata_docs.append(metadata_inter)
embeddings_metadata = model.encode(metadata_docs)

In [ ]:
# Tout est rassemblé dans une fonction
def met_match_txt(question, embeddings_metadata,k):
    query_embedding = model.encode(question)
    similarity_scores = cosine_similarity(query_embedding.reshape(1,-1), embeddings_metadata)
    # Sort the documents based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[0]
    sorted_documents = [md_header_splits[i].page_content for i in sorted_indices][::-1]
    sorted_documents=sorted_documents[:k]
    top_k_documents=""
    for i in range(k):
        top_k_documents=top_k_documents+'\n\n'+sorted_documents[i]
    return top_k_documents

In [ ]:
# Tout est rassemblé dans une fonction
def met_match_doc(question, embeddings_metadata,k):
    query_embedding = model.encode(question)
    similarity_scores = cosine_similarity(query_embedding.reshape(1,-1), embeddings_metadata)
    # Sort the documents based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[0]
    sorted_documents = [md_header_splits[i] for i in sorted_indices][::-1]
    top_k_documents=sorted_documents[:k]
    return top_k_documents

In [ ]:
met_match_txt(embeddings_metadata=embeddings_metadata,k=2,question="comment changer d'APE ?")

In [ ]:
met_match_doc(embeddings_metadata=embeddings_metadata,k=2,question="comment changer d'APE ?")

In [ ]:
# Simple chain

In [ ]:
# On crée un prompt template, on initialise le modèle et l'output parser
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
import streamlit as st
# Build prompt llama chat
template_chat = """<s>[INST] <<SYS>>
\n
Vous êtes un assistant conversationnel concis et honnête, qui répond, uniquement en langue française, aux problèmes posés par un usager. Si vous ne connaissez pas la réponse, répondez simplement que vous ne savez pas, n'essayez pas d'inventer la réponse.  
\n<</SYS>>
\n
À l'aide du contexte ci-dessous, répondez, uniquement en langue française, au problème suivant posé par un usager : "{question}". Si besoin, demandez à l'usager certaines informations afin de préciser votre réponse en fonction du contexte.
\n\n
Contexte : 
\n
{context}
[/INST]"""
QA_CHAIN_PROMPT_chat = PromptTemplate.from_template(template_chat)

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    # Verbose is required to pass to the callback manager
n_batch = 512
llm = LlamaCpp(
    model_path='./llama-2-7b-chat.Q5_K_M.gguf',
    n_gpu_layers=0,
    max_tokens = 8000,
    temperature = 0.0,
    n_batch=n_batch,
    f16_kv=True,
    use_mlock=True,
    n_ctx=2048,
    callback_manager=callback_manager,
    n_threads=8,
    verbose=True,
    streaming=True)

In [ ]:
from langchain.schema.runnable import RunnableMap
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
chain = RunnableMap({
    "context": lambda x: met_match_txt(embeddings_metadata=embeddings_metadata,k=2,question=x["question"]),
    "question": lambda x: x["question"]
}) | QA_CHAIN_PROMPT_chat | llm | output_parser

In [ ]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
# On utilise config-callbacks pour avoir le détail du déroulement de la chaîne
chain.invoke({"question": "Comment changer d'APE?"},config={'callbacks': [ConsoleCallbackHandler()]})

In [ ]:
# Ajout de la mémoire

In [ ]:
test=RunnableMap({
    "context": lambda x: met_match_txt(embeddings_metadata=embeddings_metadata,k=2,question=x["question"]),
    "question": lambda x: x["question"]}).invoke({"question": "Où puis-je obtenir mon avis de situation ?"})

In [ ]:
print(type(test))
print(test.keys())

In [ ]:
template_memory=PromptTemplate(input_variables=['chat_history', 'question','context'],
               template='''<s>[INST] <<SYS>>
\n
Vous êtes un assistant conversationnel cordial et honnête, qui répond, 
               uniquement en langue française, aux questions ou aux problèmes posés par un usager. 
               Si vous ne connaissez pas la réponse, répondez simplement que vous ne savez pas, 
               n'essayez pas d'inventer la réponse.
               \n<</SYS>>\n
               Historique de la conversation:\n{chat_history}
               \n À l'aide de l'historique de la conversation ci-dessus, et du contexte ci-dessous, répondez, 
               uniquement en langue française, au problème suivant posé par un usager : {question}\n
               [/INST]''')

In [ ]:
# On garde en mémoire l'historique des messages
# Return_messages = True signifie qu'on met les messages passés sous forme de liste, 
# et non de la forme d'un simple texte
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True)

In [ ]:
# Custom retriever à l'aide de la fonction met_match
from langchain.schema.retriever import BaseRetriever
from langchain.schema import Document
from typing import List

class CustomRetriever(BaseRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        # Use your existing retriever to get the documents
        documents = met_match_doc(embeddings_metadata=embeddings_metadata,k=2,question=query)
        
        return documents

In [ ]:
# Le module ConversationalRetrievalChain gère la mémoire

qa_memory = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=CustomRetriever(),
    memory=memory,
    condense_question_prompt=template_memory,
    verbose=True)

In [ ]:
memory_chain=RunnableMap({
    "context": lambda x: met_match_txt(embeddings_metadata=embeddings_metadata,k=2,question=x["question"]),
    "question": lambda x: x["question"]}) | qa_memory


In [ ]:
question = "Où puis-je obtenir mon avis de situation ?"
result = memory_chain.invoke({"question": question})

In [ ]:
question = "Où puis-je obtenir mon avis de situation ?"
result = qa_memory.invoke({"question": question})

In [ ]:
# Le prompt est en anglais, on le change
qa_memory_promptfr = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=CustomRetriever(),
    memory=memory,
    condense_question_prompt=template_memory,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT_chat},
    verbose=True)
#,
#    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT_chat}
# combine_docs_chain_kwargs={
        #"prompt": ChatPromptTemplate.from_messages([
            #system_message_prompt,
            #human_message_prompt,
        #]),
    #}
#)

In [ ]:
memory_chain_promptfr=RunnableMap({
    "context": lambda x: met_match_txt(embeddings_metadata=embeddings_metadata,k=2,question=x["question"]),
    "question": lambda x: x["question"]}) | qa_memory_promptfr

In [ ]:
question = "Où puis-je obtenir mon avis de situation ?"
result = qa_memory_promptfr.invoke({"question": question,'context': 'Test_contexte'})

In [ ]:
question = "Je ne sais pas si mes données sont diffusées"
result = memory_chain_promptfr.invoke({"question": question})

In [ ]:
result['answer']

In [ ]:
question = "Je ne sais pas si mes données sont diffusées"
result = qa_memory({"question": question})

In [ ]:
result['answer']